In [6]:
import numpy as np
import pandas as pd
import seaborn as sns

In [7]:
# Initialize thevariables
cols = 4
rows = 3

gamma = 0.9

R = [[-0.04,-0.04,-0.04,1],
     [-0.04,-0.04,-0.04,-1],
     [-0.04,-0.04,-0.04,-0.04]]

U = [[0,0,0,1],
     [0,0,0,-1],
     [0,0,0,0],
     [0,0,0,0]]

p_intended = 0.8
p_slip = 0.1

actions = {
    'up':    (0, -1),
    'down':  (0, 1),
    'left':  (-1, 0),
    'right': (1, 0)
}

In [8]:
U = [[0.0 for x in range(cols)] for y in range(rows)]

In [9]:
# Initialize the Terminal States
U[1][3] = -1
U[0][3] = 1

In [10]:
def in_grid(x,y):
  return 0<x<cols and 0<y<rows

In [11]:
def get_next_state(x,y,action):
  dx,dy = actions[action]

  ix,iy = dx+x, dy+y

  if not in_grid(ix,iy):
    ix,iy = x,y

  if action in ['up', 'down']:
    perp_actions = ['left', 'right']
  else:
    perp_actions = ['up', 'down']

  perp_states = []
  for perp_action in perp_actions:
    pdx, pdy = actions[perp_action]
    px, py = x + pdx, y + pdy
    if not in_grid(px, py):
        px, py = x, y
    perp_states.append((px, py))

  return [
        (p_intended, ix, iy),
        (p_slip, perp_states[0][0], perp_states[0][1]),
        (p_slip, perp_states[1][0], perp_states[1][1])
    ]

terminal_states = [(3, 0), (3, 1)]